#Dissipative Driven Entanglement Notebook
Author: Eugene Dumitrescu

In this notebook we (loosely) follow the prescription of Refs[1,2] to simulate the dynamical entanglement of QD systems coupled to a quantum environment. The approach taken is based on cavity QED, however our aim is to replace the cavity with a plasmonic reservoir.

### References
* [M. Otten, R. A. Shah, N. F. Scherer, M. Min, M. Pelton, and S. K. Gray, Phys. Rev. B 92, 125432 (2015)](http://journals.aps.org/prb/abstract/10.1103/PhysRevB.92.125432)
* [Dissipation-driven entanglement between qubits mediated by plasmonic nanoantennas](http://journals.aps.org/prb/abstract/10.1103/PhysRevB.89.235413)


In [23]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from qutip import *
import scipy.signal as ss

# operators acting on the composite spin-boson Hilbert space
def I(N,M):
    """Identity"""
    oplist = [qeye(2)] * N
    oplist.append(qeye(M))
    return tensor(oplist)

# spin operators
def sx(i, N, M):
    """Pauli X on qb i"""
    oplist = [qeye(2)] * N
    oplist[i] = sigmax()
    oplist.append(qeye(M))
    return tensor(oplist)

def sy(i, N, M):
    """Pauli Y on qb i"""
    oplist = [qeye(2)] * N
    oplist[i] = sigmay()
    oplist.append(qeye(M))
    return tensor(oplist)

def sz(i, N, M):
    """Pauli Z on qb i"""
    oplist = [qeye(2)] * N
    oplist[i] = sigmaz()
    oplist.append(qeye(M))
    return tensor(oplist)

def sm(i, N, M):
    """destroy quanta on qb i"""
    oplist = [qeye(2)] * N
    oplist[i] = sigmam()
    oplist.append(qeye(M))
    return tensor(oplist)

def sp(i, N, M):
    """create quanta on qb i"""
    oplist = [qeye(2)] * N
    oplist[i] = sigmap()
    oplist.append(qeye(M))
    return tensor(oplist)

def d(i,N,M):
    """dipole operator for ith spin"""
    return sp(i, N, M) + sm(i, N, M)

# plasmon / EM mode operators
def a(N, M):
    """bosonic annihiliation operator"""
    oplist = [qeye(2)] * N
    oplist.append(destroy(M))
    return tensor(oplist)

def n_a(N,M):
    """bosonic number operator"""
    return a(N,M).dag() * a(N,M)

def d_a(N,M):
    """bosonic dipole operator"""
    return a(N,M).dag() + a(N,M)

# spin subspace projection operators 
def P_S(N,M):
    """Symmetric |s = 1, m_l = 0> subspace projector"""
    return .25 * (I(N,M) - sz(0,N,M) * sz(1,N,M) + sx(0,N,M) * sx(1,N,M) + sy(0,N,M) * sy(1,N,M))

def P_A(N,M):
    """Anti-symmetric |s = 0, m_l = 0> subspace projector"""
    return .25 * (I(N,M) - sz(0,N,M) * sz(1,N,M) - sx(0,N,M) * sx(1,N,M) - sy(0,N,M) * sy(1,N,M))

def P_pp(N,M):
    """|++> subspace projector """
    return np.prod([sp(i,N,M) * sm(i,N,M)  for i in range(N)])

def P_mm(N,M):
    """|--> subspace projector """
    return np.prod([sm(i,N,M) * sp(i,N,M)  for i in range(N)])


## Hamiltonian and Model

Our model Hamiltonian for a N two levels systems (TLS) interacting with a bosonic reservoir is 
<center> $H = H_0 + H_{int} + H_{drv}$ </center>
where the individual, interacting, and driving components of the Hamiltonian are given by 
<center>
$\displaystyle H_0 = - \omega_0 \sum_{i=1}^N \sigma_z^{(i)} + \omega_c a^\dagger a$
</center>
<center>
$\displaystyle H_{int} = - \sum_{i}^N g_i (\sigma^{+}a + \sigma^{-}a^\dagger)$
</center>
<center>
$\displaystyle H_{drv} = - \sum_{i}^N \Omega_i \hat{d}_i + \Omega_a \hat{d}_a$
</center>
We have used the EM field and TLS dipole operators $\hat{d}_i = \sigma^+_{(i)}+\sigma^-_{(i)}$ and $\hat{d}_a = a + a^\dagger$  in the driving term and we have also applied the rotating wave approximation.

In [2]:
N = 2          # Number of spin 1/2's
M = 2          # Number of cavity modes
lam = 2        # constant that all other constants are given in terms of

ws = [0.020, -0.020, 0.0]    # frequency/level spacing of each individual system
gs = [0.1, 0.1]              # position dependent couplings
ds = [0.1, 0.1, 0]           # driving strengths qb0,1, bosons 

def Ham(N,M,oms,coups,drvgs):
    """ Define the Hamiltonian as a sum over:

    Terms
    -----
    oms: 
    coups:
    drvgs: 
    
    Returns
    -------
    H_0: non-interacting systems
    H_RWA: interaction between subsystems in the rotating wave approximation
    H_DRV: driving terms in the co-rotating reference frame

    Hermitial Hamiltonian operator
    H_0 - H_RWA - H_DRV
    """
    H_0   = sum(oms[i] * sp(i,N,M) * sm(i,N,M) for i in range(N)) + oms[N] * n_a(N,M)
    H_RWA = sum(coups[i] * (a(N,M) * sp(i,N,M) + a(N,M).dag() * sm(i,N,M)) for i in range(N))
    H_DRV = sum(drvgs[i] * d(i,N,M) for i in range(N)) + drvgs[N] * d_a(N,M)
    return H_0 - H_RWA - H_DRV

# Lindblad dissipation parameters
kap   = 1 * lam           # cavity relaxation rate
gam   = 1 * kap / 1e8     # atom relaxation rate
gdph  = 2 * kap / 1e8     # atom dephasing rate 
n_th  = 0.0               # thermal cavity occupation number 

def cops(kappa, gamma, deph, n_th):
    """
    Define collapse operators for Lindblad master equation
    
    Parameters
    ----------
    kappa: cavity relaxation rate,  ~ 500 THz ~ 2 eV
    gamma: emitter relaxation rate, ~ 10 MHz ~ 1mueV
    gdph: emitter dephasing rate,   ~ same as relaxation rate
    n_th: temperature for entire system, gives excitation rate

    Returns
    -------
    c_ops: list of collapse operators
    """
    c_ops = []
    for i in range(2):
        rate = np.sqrt(gamma * (1 + n_th))
        if rate > 0.0:
            c_ops.append(rate * sm(i,N,M))        # relaxation
        rate = gamma * n_th
        if rate > 0.0:
            c_ops.append(rate * sm(i,N,M).dag())  # excitation
        c_ops.append(np.sqrt(deph) * sz(i,N,M))   # dephasing  
    rate = np.sqrt(kappa * (n_th + 1))
    if rate > 0: 
        c_ops.append(rate * a(N,M))
    rate = np.sqrt(kappa * (n_th))
    if rate > 0: 
        c_ops.append(rate * a(N,M).dag())
    return c_ops

def g2(sta, ham, lin, tim):
    """
    Parameters
    ----------
    sta: initial state to evolve according to 
    ham: Hamiltonian matrix and 
    lin: Limbladian operators for 
    tim: times 
    
    Returns
    -------
    g^(2)(tau) second order correlation function
    """
    e_ss_ops = [sp(i,N,M) * sm(i,N,M) for i in range(N)]        # qubit occupation number operators
    
    rho_ae = [sm(i,N,M) * sta * sp(i,N,M) for i in range(N)]    # track all possible emission events
#   rho_ae = [r/r.tr() for r in rho_ae]                         # renormalize states
    
    # solve for <sp*sm> with respect to the post emission state
    results = [mesolve(ham, r, taus, lin, e_ss_ops) for r in rho_ae]
    
    # steady state g2 denominator/normalization constants
    denoms = [np.real((e * sta).tr()) for e in e_ss_ops]  
    return sum(results[i].expect[j] / (denoms[i] * denoms[j]) / float(N**2) for i in range(N) for j in range(N))


def freq(pwrspec, o = 1):
    """
    Extract the primary non-decay frequency from the 
    Parameters
    ----------
    pwrspec: g2 power spectrum
    order: number of points frequencym must be above to count as local maxima    
    Returns
    -------
    two qubit 'Rabi' frequency -- lowest one which emerges.... should probably be the biggest?
    """
    
    fs = ss.argrelmax(pwrspec[0:int(steps/2)], order = o) # candidate characteristic frequencies
    M = [pwrspec[f] for f in fs][0]                       # power spectrum magnitude of frequencies 
    return fs[0][np.ndarray.argmax(M)]

In [24]:
[0.2, -0.2, .0001][2]

0.0001

In [34]:
print(ptrace(Ham(2,3,[0.2, -0.2, -.001],[-2000, 2000],[0.0, 0.0, 0]),[0,1]))

print(ptrace(Ham(2,3,[0.2, -0.2, -.001],[0, 0],[0.0, 0.0, 0]),[0,1]))

# ws = [0.020, 0.020, 0.0]     # frequency/level spacing of each individual system
# gs = [0.1, 0.1]              # position dependent couplings
# ds = [0.1, 0.1, 0]           # driving strengths qb0,1, bosons 
# print(ptrace(Ham(ws,gs,ds),[0,1]))

Quantum object: dims = [[2, 2], [2, 2]], shape = [4, 4], type = oper, isherm = True
Qobj data =
[[-0.003  0.     0.     0.   ]
 [ 0.     0.597  0.     0.   ]
 [ 0.     0.    -0.603  0.   ]
 [ 0.     0.     0.    -0.003]]
Quantum object: dims = [[2, 2], [2, 2]], shape = [4, 4], type = oper, isherm = True
Qobj data =
[[-0.003  0.     0.     0.   ]
 [ 0.     0.597  0.     0.   ]
 [ 0.     0.    -0.603  0.   ]
 [ 0.     0.     0.    -0.003]]


In [32]:
Ham(2,3,[0.2, -0.2, 1],[5, 5],[0.0, 0.0, 0])

Quantum object: dims = [[2, 2, 3], [2, 2, 3]], shape = [12, 12], type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.         -5.          0.          0.
  -5.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.         -7.07106781
   0.          0.         -7.07106781  0.          0.          0.        ]
 [ 0.          0.          2.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.2         0.          0.          0.
   0.          0.          0.         -5.          0.        ]
 [-5.          0.          0.          0.          1.2         0.          0.
   0.          0.          0.          0.         -7.07106781]
 [ 0.         -7.07106781  0.          0.          0.          2.2         0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0. 

In [35]:
Ham(2,3,[0.2, -0.2, 1],[2, 2],[0.0, 0.0, 0])

Quantum object: dims = [[2, 2, 3], [2, 2, 3]], shape = [12, 12], type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.         -2.          0.          0.
  -2.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.         -2.82842712
   0.          0.         -2.82842712  0.          0.          0.        ]
 [ 0.          0.          2.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.2         0.          0.          0.
   0.          0.          0.         -2.          0.        ]
 [-2.          0.          0.          0.          1.2         0.          0.
   0.          0.          0.          0.         -2.82842712]
 [ 0.         -2.82842712  0.          0.          0.          2.2         0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0. 

In [9]:
Ham(2,2,[0.2, -0.2, 0],[2, 2],[0.0, 0.0, 0])

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = [8, 8], type = oper, isherm = True
Qobj data =
[[ 0.   0.   0.  -2.   0.  -2.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.2  0.   0.   0.   0.  -2. ]
 [-2.   0.   0.   0.2  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -0.2  0.   0.  -2. ]
 [-2.   0.   0.   0.   0.  -0.2  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -2.   0.  -2.   0.   0.   0. ]]